In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [5]:
fileName = 'https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/2gaussian.txt'

data = np.loadtxt(fileName)

print(data.shape)

(6000, 2)


In [18]:
class GaussianEM:
    def __init__(self,epochs = 100,gaussians = 2):
        self.epochs = epochs
        self.gaussians = gaussians
    
    def initParams(self,X):
        dataPoints, dims = X.shape
        self.cov = [[0 for _ in range(dims)] for _ in range(dims)]
        self.mu = [0 for _ in range(dims)]
        self.selectProb = [1]*self.gaussians
        self.selectProb = [x/self.gaussians for x in self.selectProb]
    
    def multivariateGaussian(self,x,mu,cov):
        d = len(mu)
        coeff = 1 / ((2 * np.pi) ** (d / 2) * np.linalg.det(cov) ** 0.5)
        x_mu = x - mu
        exponent = -0.5 * np.dot(np.dot(x_mu, np.linalg.inv(cov)), x_mu.T)
        return coeff * np.exp(exponent)
    
    def assignPointsResponsibility(self,X):
        dataPoints, dims = X.shape
        self.responsibility = np.zeros((dataPoints, self.gaussians))

        for i in range(dataPoints):
            for j in range(self.gaussians):
                self.responsibility[i][j] = self.selectProb[j] * self.multivariateGaussian(X[i], self.mu[j], self.cov[j])
            self.responsibility[i] /= np.sum(self.responsibility[i])

    def updateParams(self,X):
        dataPoints, dims = X.shape
        for j in range(self.gaussians):
            N_j = np.sum(self.responsibility[:, j])
            self.mu[j] = np.sum(self.responsibility[:, j][:, np.newaxis] * X, axis=0) / N_j
            diff = X - self.mu[j]
            self.cov[j] = np.dot((self.responsibility[:, j][:, np.newaxis] * diff).T, diff) / N_j
            self.selectProb[j] = N_j / dataPoints


    def fit(self,X):
        self.initParams(X)
        print(self.selectProb)
        print(self.mu)
        print(self.cov)

        for i in range(self.epochs):
            self.assignPointsResponsibility(X)
            self.updateParams(X)

        return self.mu, self.cov, self.selectProb


In [19]:
gmm = GaussianEM(100)
mu,cov,select = gmm.fit(data)

print(mu,cov,select)

[0.5, 0.5]
[0, 0]
[[0, 0], [0, 0]]


TypeError: object of type 'int' has no len()